### IMPORTS

In [1]:
import ipynb.fs.full.processing as processing
import ipynb.fs.full.training as training
import ipynb.fs.full.analysis as analysis
import ipynb.fs.full.storage as storage
import ipynb.fs.full.visualize as visualize
import ipynb.fs.full.misc as misc

x == foobar


In [2]:
import ipynb.fs.full.splitting as splitting
import ipynb.fs.full.table as table
import ipynb.fs.full.decide as decide

In [3]:
import ipynb.fs.full.real_features as features

In [4]:
import numpy as np

### SERIALIZE THE YAML CONFIG

In [5]:
config = storage.load_yaml('extra/config.yaml')

In [6]:
config

{'data': 'extra/dataset.csv',
 'processing': {'resample': True,
  'time': 'D',
  'aggregate': {'Open': 'first',
   'High': 'max',
   'Low': 'min',
   'Close': 'last',
   'Volume': 'sum'},
  'label': {'from': 'Close', 'shift': 1}},
 'features': {'category': 'all', 'window': 14},
 'splitting': {'train_split': 0.8, 'validation_folds': 5},
 'regression_ensemble': {'models': [{'linreg': None},
   {'lstm': {'morph': {'window': 4, 'batch': 30},
     'layers': [{'lstm': {'units': 120}},
      {'dropout': {'rate': 0.15}},
      {'dense': {'units': 50, 'activation': 'relu'}},
      {'dense': {'units': 1}}],
     'epochs': 15,
     'loss': 'mean_squared_error',
     'optimizer': 'rmsprop'}}]},
 'classification_ensemble': {'decision': {'name': 'quantile',
   'upper': 0.75,
   'lower': 0.25},
  'models': [{'logreg': None}, {'rand_forest': None}]}}

### STEP 1: PROCESS RAW DATA TO DATAFRAME

In [7]:
dataframe = processing.create_dataframe(config)

In [8]:
dataframe.head(5)

,Open,High,Low,Close,Volume,label
Date_Timestamp,,,,,,
2010-01-01,1.43327,1.43356,1.43207,1.43335,39761.000053,1.43141
2010-01-03,1.43024,1.43359,1.42951,1.43141,3001.600003,1.44244
2010-01-04,1.43143,1.44556,1.42559,1.44244,80019.400094,1.43634
2010-01-05,1.44238,1.44834,1.43445,1.43634,79887.100067,1.44005
2010-01-06,1.43638,1.44342,1.42807,1.44005,80971.800085,1.43155


### STEP 2: ADD FEATURES TO DATAFRAME

In [9]:
regression_dataset = features.add(dataframe, config['features'])

In [10]:
regression_dataset.head(5)

,open,high,low,close,volume,label,stock,stocd,stocsd,momentum,...,rsi,obv,ma,bias6,psy12,asy5,asy4,asy3,asy2,asy1
Date_Timestamp,,,,,,,,,,,,,,,,,,,,,
2010-01-21,1.41115,1.41422,1.40247,1.40979,78980.100041,1.41361,13.189189,9.578919,19.849579,-0.02655,...,29.873956,-159980.600046,1.425180,-0.012336,41.7,-0.403064,-0.471831,-0.715772,-0.637503,-0.099256
2010-01-22,1.40977,1.41805,1.40837,1.41361,76660.300037,1.41517,20.072072,13.339429,14.445538,-0.02644,...,33.983332,-83320.300009,1.420572,-0.006774,41.7,-0.323346,-0.469180,-0.334804,0.085670,0.270596
2010-01-24,1.41557,1.41705,1.41396,1.41517,3128.600004,1.41458,22.882883,18.714715,13.877688,-0.01638,...,35.641946,-80191.700005,1.415528,-0.003171,33.3,-0.353285,-0.223529,0.093878,0.190445,0.110295
2010-01-25,1.41498,1.41939,1.41248,1.41458,79577.200048,1.40808,21.819820,21.591592,17.881912,-0.02625,...,35.280916,-159768.900053,1.412868,-0.000558,33.3,-0.187163,0.059984,0.113064,0.034298,-0.041700
2010-01-26,1.41454,1.41780,1.40397,1.40808,79232.600053,1.40280,10.108108,18.270270,19.525526,-0.03823,...,31.495786,-239001.500106,1.412246,-0.002826,25.0,-0.044125,-0.030342,-0.130655,-0.251130,-0.460559


### STEP 3: SPLIT & SCALE PRIMARY DATASET

In [11]:
features = regression_dataset.loc[:, regression_dataset.columns != 'label'].to_numpy()

In [12]:
labels = regression_dataset[['label']].to_numpy()

In [13]:
primary, scaler = splitting.primary(features, labels, config['splitting']['train_split'])

### STEP 4: CREATE CROSS VALIDATION FOLDS & TRAIN REGRESSION MODELS

In [14]:
regression_ensemble = [None] * len(config['regression_ensemble']['models'])

In [15]:
# LOOP THROUGH REQUESTED MOEDLS
for index, item in enumerate(config['regression_ensemble']['models']):
    
    # MODEL PROPS
    name = list(item)[0]
    settings = item[name]
    
    # CONTAINER
    folds = []
    
    # IF THE MODEL HAS EXTRA SETTINGS
    if settings:
        folds = splitting.timeseries(
            primary['train'],
            config['splitting']['validation_folds'],
            window=settings['morph']['window']
        )
    
    # OTHERWISE
    else:
        folds = splitting.timeseries(
            primary['train'],
            config['splitting']['validation_folds']
        )
        
    # FOLD PREDICTIONS
    temp_predictions = []
    temp_labels = []
    
    # TRAIN & PREDICT WITH EACH FOLD
    for fold in folds:
        predictions = training.start(fold, name, settings)
        
        # APPEND TO COLLECTIONS
        temp_predictions.append(predictions)
        temp_labels.append(fold['test']['labels'])
        
    # APPEND RESULTS TO ENSEMBLE
    regression_ensemble[index] = {
        'name': name + '_' + str(index),
        'predictions': np.concatenate(temp_predictions),
        'labels': np.concatenate(temp_labels)
    }

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### STEP 5: CONSTRUCT TABLE FOR ENSEMBLE PREDICTIONS

In [16]:
ensemble_table = table.ensemble(regression_ensemble)

In [17]:
ensemble_table.head(10)

,linreg_0,lstm_1,label
0,1.415415,1.290739,1.40686
1,1.411078,1.281746,1.41436
2,1.419329,1.279866,1.43163
3,1.433975,1.284370,1.42894
4,1.422772,1.274651,1.43328
5,1.430989,1.272738,1.44240
6,1.441615,1.268324,1.43448
7,1.432478,1.272735,1.44864
8,1.451175,1.283896,1.46348
9,1.465617,1.289327,1.46522


### STEP 6: CLASSIFY LABELS WITH REQUESTED METHOD

In [18]:
# BUY   = 0
# SELL  = 1
# HOLD  = 2

In [19]:
classification_dataset = decide.label({
    'predictions': ensemble_table,
    'settings': config['classification_ensemble']['decision']
})

In [20]:
classification_dataset.head(25)

,linreg_0,lstm_1,label
0,1.415415,1.290739,2
1,1.411078,1.281746,1
2,1.419329,1.279866,1
3,1.433975,1.284370,2
4,1.422772,1.274651,1
5,1.430989,1.272738,1
6,1.441615,1.268324,0
7,1.432478,1.272735,1
8,1.451175,1.283896,1
9,1.465617,1.289327,2


#### DECISION BREAKDOWN

In [21]:
classification_dataset['label'].value_counts(normalize=True)

2    0.500483
1    0.249758
0    0.249758
Name: label, dtype: float64